In [1]:

import tensorflow as tf
print(tf.__version__)



2.15.0


In [2]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [5]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [6]:
from keras.layers import Input


In [7]:
from tensorflow.keras.layers import Input
import tensorflow as tf
import pathlib
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from numpy import zeros, ones
from numpy.random import randint
from matplotlib import pyplot as plt


In [8]:
import numpy as np
import os
from skimage.transform import resize
from matplotlib import pyplot as plt

# Define file paths
input_dir = r"C:\Users\net pc\Desktop\Thesis Paper 3\Brain tumor dataset\Training\pituitary_tumor_inputs"
label_dir = r"C:\Users\net pc\Desktop\Thesis Paper 3\Brain tumor dataset\Training\pituitary_tumor_targets"


def load_data(input_dir, label_dir, target_shape):
    input_files = os.listdir(input_dir)
    label_files = os.listdir(label_dir)
    
    # Sort files to ensure corresponding inputs and labels match
    input_files.sort()
    label_files.sort()
    
    trainA = []
    trainB = []
    for i in range(len(input_files)):
        input_img = plt.imread(os.path.join(input_dir, input_files[i]))
        label_img = plt.imread(os.path.join(label_dir, label_files[i]))
        
        # Resize images to target shape if they have different shapes
        if input_img.shape != target_shape:
            input_img = resize(input_img, target_shape)
        if label_img.shape != target_shape:
            label_img = resize(label_img, target_shape)
        
        trainA.append(input_img)
        trainB.append(label_img)
    
    return np.array(trainA), np.array(trainB)



# Define target shape for images (e.g., (256, 256, 3))
target_shape = (256, 256, 3)

# Load data
data = load_data(input_dir, label_dir, target_shape)
print("Data loaded successfully!")


Data loaded successfully!


In [9]:


# Define discriminator
def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape, name='input_image')
    in_target_image = Input(shape=image_shape, name='target_image')
    merged = Concatenate()([in_src_image, in_target_image])
    
    # Discriminator architecture
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(patch_out)
    
    model = Model([in_src_image, in_target_image], patch_out)
    model.compile(loss='binary_crossentropy', optimizer='adam', loss_weights=[0.5])
    return model

In [10]:
def define_generator(image_shape=(256,256,3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    
    # Encoder
    e1 = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    e2 = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e1))
    e2 = BatchNormalization()(e2)
    e3 = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e2))
    e3 = BatchNormalization()(e3)
    e4 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e3))
    e4 = BatchNormalization()(e4)
    e5 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e4))
    e5 = BatchNormalization()(e5)
    e6 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e5))
    e6 = BatchNormalization()(e6)
    e7 = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e6))
    e7 = BatchNormalization()(e7)
    
    # Bottleneck
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e7))
    b = Activation('relu')(b)
    
    # Decoder
    d1 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(b))
    d1 = BatchNormalization()(d1)
    d1 = Dropout(0.5)(d1)
    d1 = Concatenate()([d1, e7])
    d2 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d1))
    d2 = BatchNormalization()(d2)
    d2 = Dropout(0.5)(d2)
    d2 = Concatenate()([d2, e6])
    d3 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d2))
    d3 = BatchNormalization()(d3)
    d3 = Concatenate()([d3, e5])
    d4 = Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d3))
    d4 = BatchNormalization()(d4)
    d4 = Concatenate()([d4, e4])
    d5 = Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d4))
    d5 = BatchNormalization()(d5)
    d5 = Concatenate()([d5, e3])
    d6 = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d5))
    d6 = BatchNormalization()(d6)
    d6 = Concatenate()([d6, e2])
    d7 = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('relu')(d6))
    d7 = BatchNormalization()(d7)
    d7 = Concatenate()([d7, e1])
    
    # Output
    g = Conv2DTranspose(image_shape[2], (4,4), strides=(2,2), padding='same', kernel_initializer=init)(Activation('tanh')(d7))
    out_image = Activation('tanh')(g)
    
    model = Model(in_image, out_image)
    return model


In [11]:
def define_gan(g_model, d_model, image_shape):
    # Make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False

    # Define the source image
    in_src = Input(shape=image_shape)
    
    # Supply the image as input to the generator 
    gen_out = g_model(in_src)
    
    # Supply the input image and generated image as inputs to the discriminator
    dis_out = d_model([in_src, gen_out])
    
    # Combine source image input, generated image, and discriminator output into a model
    model = Model(in_src, [dis_out, gen_out])
    
    return model



In [12]:


def define_combined_model(g_model, d_model, image_shape):
    # Define the source image
    in_src = Input(shape=image_shape)
    
    # Supply the image as input to the generator 
    gen_out = g_model(in_src)
    
    # Supply the input image and generated image as inputs to the discriminator
    dis_out = d_model([in_src, gen_out])
    
    # Combine source image input, generated image, and discriminator output into a model
    model = Model(in_src, [dis_out, gen_out])
    
    # Define GAN model
    gan_model = define_gan(g_model, d_model, image_shape)

    # Compile the GAN model
    opt = Adam(lr=0.0002, beta_1=0.5)
    gan_model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1, 100])

    return model


In [13]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

In [14]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y


In [15]:

# generate samples and save as a plot and save the model
#GAN models do not converge, we just want to find a good balance between
#the generator and the discriminator. Therefore, it makes sense to periodically
#save the generator model and check how good the generated image looks. 


import os
from keras.preprocessing.image import save_img

def summarize_performance(step, g_model, dataset, n_samples=3):
    print("Saving images...")
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    
    output_path = "C:\\Users\\net pc\\Desktop\\Thesis Paper 3\\pituitary_tumor_generated_Pix2Pix"
    
    for i in range(n_samples):
        # save real source image as JPEG
        real_source_image = X_realA[i]
        real_source_filename = os.path.join(output_path, f'real_source_{step}_{i + 1}.jpeg')
        save_img(real_source_filename, real_source_image)

        # save generated target image as JPEG
        generated_target_image = X_fakeB[i]
        generated_target_filename = os.path.join(output_path, f'generated_target_{step}_{i + 1}.jpeg')
        save_img(generated_target_filename, generated_target_image)

        # save real target image as JPEG
        real_target_image = X_realB[i]
        real_target_filename = os.path.join(output_path, f'real_target_{step}_{i + 1}.jpeg')
        save_img(real_target_filename, real_target_image)

    # save the generator model
    model_filename = os.path.join(output_path, f'model_{step}.h5')
    g_model.save(model_filename)

    print(f"Images and model saved at {output_path}")



In [16]:

# train pix2pix models
def train(d_model, g_model, gan_model, dataset, n_epochs=10000, n_batch=1):
	# determine the output square shape of the discriminator
	n_patch = d_model.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
		# update discriminator for real samples
		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		# update discriminator for generated samples
		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		# update the generator
		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
		# summarize performance
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
		# summarize model performance
		if (i+1) % (bat_per_epo * 10) == 0:
			summarize_performance(i, g_model, dataset)


In [17]:
# Define discriminator model
d_model = define_discriminator(target_shape)

# Define generator model
g_model = define_generator(target_shape)

# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)

# Define combined model
combined_model = define_combined_model(g_model, d_model, target_shape)



C:\Users\Public\Efendim\Lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [ ]:
# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)

# Compile the GAN model
opt = Adam(lr=0.0002, beta_1=0.5)
gan_model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1, 100])

# Train Pix2Pix models using the GAN model
train(d_model, g_model, gan_model, data, n_epochs=10000, n_batch=1)


1/1 [==============================] - 2s 2s/step



>1, d1[0.900] d2[2.019] g[23.287]
1/1 [==============================] - 0s 258ms/step
>2, d1[1.619] d2[2.414] g[20.222]
1/1 [==============================] - 0s 374ms/step
>3, d1[0.362] d2[1.192] g[19.800]
1/1 [==============================] - 0s 466ms/step
>4, d1[0.410] d2[0.881] g[16.952]
1/1 [==============================] - 0s 415ms/step
>5, d1[0.299] d2[0.744] g[13.986]
1/1 [==============================] - 0s 317ms/step
>6, d1[0.453] d2[0.336] g[12.124]
1/1 [==============================] - 0s 249ms/step
>7, d1[0.533] d2[0.271] g[11.960]
1/1 [==============================] - 0s 224ms/step
>8, d1[0.589] d2[0.250] g[10.823]
1/1 [==============================] - 0s 225ms/step
>9, d1[0.593] d2[0.257] g[19.591]
1/1 [==============================] - 0s 205ms/step
>10, d1[0.433] d2[0.384] g[14.772]
1/1 [==============================] - 0s 218ms/step
>11, d1[0.362] d2[0.406] g[11.740]
1/1 [==============================] - 0s 179ms/step
>12, d1[0.359] d2[0.523] g[12.335]
1/1 [=

1/1 [==============================] - 0s 181ms/step
>95, d1[0.080] d2[0.055] g[13.934]
1/1 [==============================] - 0s 175ms/step
>96, d1[0.370] d2[0.035] g[11.947]
1/1 [==============================] - 0s 191ms/step
>97, d1[0.073] d2[0.070] g[8.891]
1/1 [==============================] - 0s 170ms/step
>98, d1[0.112] d2[0.071] g[7.678]
1/1 [==============================] - 0s 136ms/step
>99, d1[0.776] d2[0.188] g[15.842]
1/1 [==============================] - 0s 117ms/step
>100, d1[0.088] d2[0.327] g[8.660]
1/1 [==============================] - 0s 144ms/step
>101, d1[0.055] d2[0.443] g[10.086]
1/1 [==============================] - 0s 148ms/step
>102, d1[0.047] d2[0.179] g[8.606]
1/1 [==============================] - 0s 120ms/step
>103, d1[0.043] d2[0.101] g[8.845]
1/1 [==============================] - 0s 115ms/step
>104, d1[0.493] d2[0.073] g[8.003]
1/1 [==============================] - 0s 157ms/step
>105, d1[0.187] d2[0.114] g[7.618]
1/1 [============================

1/1 [==============================] - 0s 185ms/step
>188, d1[0.007] d2[0.060] g[12.923]
1/1 [==============================] - 0s 199ms/step
>189, d1[0.028] d2[0.187] g[8.531]
1/1 [==============================] - 0s 146ms/step
>190, d1[0.606] d2[0.038] g[14.514]
1/1 [==============================] - 0s 166ms/step
>191, d1[0.165] d2[0.069] g[7.180]
1/1 [==============================] - 0s 124ms/step
>192, d1[0.211] d2[0.081] g[15.305]
1/1 [==============================] - 0s 184ms/step
>193, d1[0.046] d2[0.116] g[9.734]
1/1 [==============================] - 0s 185ms/step
>194, d1[0.011] d2[0.167] g[10.007]
1/1 [==============================] - 0s 181ms/step
>195, d1[0.057] d2[0.245] g[15.379]
1/1 [==============================] - 0s 184ms/step
>196, d1[0.064] d2[0.103] g[9.141]
1/1 [==============================] - 0s 182ms/step
>197, d1[0.108] d2[0.051] g[10.333]
1/1 [==============================] - 0s 130ms/step
>198, d1[0.408] d2[0.093] g[13.708]
1/1 [====================

1/1 [==============================] - 1s 617ms/step
>280, d1[0.402] d2[0.179] g[10.904]
1/1 [==============================] - 1s 572ms/step
>281, d1[0.276] d2[0.441] g[8.086]
1/1 [==============================] - 1s 547ms/step
>282, d1[0.102] d2[0.448] g[9.600]
1/1 [==============================] - 1s 717ms/step
>283, d1[0.039] d2[0.370] g[8.869]
1/1 [==============================] - 0s 442ms/step
>284, d1[0.300] d2[0.273] g[6.760]
1/1 [==============================] - 1s 623ms/step
>285, d1[0.422] d2[0.257] g[6.436]
1/1 [==============================] - 1s 627ms/step
>286, d1[0.190] d2[0.246] g[8.460]
1/1 [==============================] - 1s 626ms/step
>287, d1[0.432] d2[0.354] g[9.924]
1/1 [==============================] - 1s 618ms/step
>288, d1[0.374] d2[0.380] g[8.638]
1/1 [==============================] - 0s 491ms/step
>289, d1[0.313] d2[0.272] g[16.151]
1/1 [==============================] - 1s 591ms/step
>290, d1[0.313] d2[0.324] g[9.990]
1/1 [=========================

...

Images and model saved at C:\Users\net pc\Desktop\Thesis Paper 3\pituitary_tumor_generated_Pix2Pix
1/1 [==============================] - 1s 971ms/step
>11101, d1[0.342] d2[0.461] g[2.935]
1/1 [==============================] - 0s 134ms/step
>11102, d1[0.025] d2[0.253] g[6.364]
1/1 [==============================] - 0s 281ms/step
>11103, d1[0.071] d2[0.261] g[4.764]
1/1 [==============================] - 0s 198ms/step
>11104, d1[0.326] d2[0.142] g[1.823]
1/1 [==============================] - 0s 166ms/step
>11105, d1[0.227] d2[0.176] g[1.605]
1/1 [==============================] - 0s 247ms/step
>11106, d1[0.131] d2[0.220] g[4.897]
1/1 [==============================] - 0s 204ms/step
>11107, d1[0.105] d2[0.385] g[9.104]
1/1 [==============================] - 0s 185ms/step
>11108, d1[0.511] d2[0.192] g[2.719]
1/1 [==============================] - 0s 181ms/step
>11109, d1[0.275] d2[0.151] g[7.016]
1/1 [==============================] - 0s 214ms/step
>11110, d1[0.169] d2[0.299] g[9.573]
1

1/1 [==============================] - 0s 213ms/step
>11191, d1[0.226] d2[0.260] g[1.850]
1/1 [==============================] - 0s 269ms/step
>11192, d1[0.288] d2[0.290] g[3.449]
1/1 [==============================] - 0s 352ms/step
>11193, d1[0.346] d2[0.293] g[4.265]
1/1 [==============================] - 0s 363ms/step
>11194, d1[0.424] d2[0.160] g[2.040]
1/1 [==============================] - 0s 274ms/step
>11195, d1[0.301] d2[0.389] g[6.835]
1/1 [==============================] - 0s 266ms/step
>11196, d1[0.168] d2[0.279] g[3.065]
1/1 [==============================] - 0s 249ms/step
>11197, d1[0.148] d2[0.237] g[4.155]
1/1 [==============================] - 0s 212ms/step
>11198, d1[0.131] d2[0.178] g[1.583]
1/1 [==============================] - 0s 227ms/step
>11199, d1[0.200] d2[0.241] g[6.133]
1/1 [==============================] - 0s 204ms/step
>11200, d1[0.113] d2[0.311] g[3.968]
1/1 [==============================] - 0s 246ms/step
>11201, d1[0.144] d2[0.408] g[1.704]
1/1 [=====

1/1 [==============================] - 0s 213ms/step
>11282, d1[0.467] d2[0.210] g[9.225]
1/1 [==============================] - 0s 209ms/step
>11283, d1[0.451] d2[0.303] g[5.652]
1/1 [==============================] - 0s 206ms/step
>11284, d1[0.263] d2[0.109] g[3.705]
1/1 [==============================] - 0s 221ms/step
>11285, d1[0.539] d2[0.197] g[4.236]
1/1 [==============================] - 0s 257ms/step
>11286, d1[0.537] d2[0.326] g[3.233]
1/1 [==============================] - 0s 223ms/step
>11287, d1[0.124] d2[0.923] g[6.003]
1/1 [==============================] - 0s 226ms/step
>11288, d1[0.056] d2[0.323] g[3.008]
1/1 [==============================] - 0s 229ms/step
>11289, d1[0.098] d2[0.528] g[3.691]
1/1 [==============================] - 0s 235ms/step
>11290, d1[0.135] d2[0.264] g[5.196]
1/1 [==============================] - 0s 216ms/step
>11291, d1[0.261] d2[0.409] g[9.757]
1/1 [==============================] - 0s 256ms/step
>11292, d1[0.231] d2[0.152] g[2.975]
1/1 [=====

1/1 [==============================] - 0s 264ms/step
>11373, d1[0.098] d2[0.106] g[5.914]
1/1 [==============================] - 0s 280ms/step
>11374, d1[0.452] d2[0.275] g[2.538]
1/1 [==============================] - 0s 222ms/step
>11375, d1[0.233] d2[0.124] g[2.781]
1/1 [==============================] - 0s 246ms/step
>11376, d1[0.280] d2[0.094] g[1.565]
1/1 [==============================] - 0s 261ms/step
>11377, d1[0.128] d2[0.333] g[4.058]
1/1 [==============================] - 0s 220ms/step
>11378, d1[0.262] d2[0.589] g[6.394]
1/1 [==============================] - 0s 247ms/step
>11379, d1[0.245] d2[0.234] g[5.804]
1/1 [==============================] - 0s 219ms/step
>11380, d1[0.101] d2[0.527] g[9.211]
1/1 [==============================] - 0s 270ms/step
>11381, d1[0.102] d2[0.137] g[8.169]
1/1 [==============================] - 0s 234ms/step
>11382, d1[0.278] d2[0.087] g[2.875]
1/1 [==============================] - 0s 193ms/step
>11383, d1[0.647] d2[0.078] g[9.433]
1/1 [=====

1/1 [==============================] - 0s 193ms/step
>11464, d1[0.266] d2[0.138] g[4.311]
1/1 [==============================] - 0s 249ms/step
>11465, d1[0.497] d2[0.284] g[2.203]
1/1 [==============================] - 0s 253ms/step
>11466, d1[0.256] d2[0.305] g[8.023]
1/1 [==============================] - 0s 257ms/step
>11467, d1[0.086] d2[0.360] g[2.907]
1/1 [==============================] - 0s 261ms/step
>11468, d1[0.262] d2[0.337] g[3.208]
1/1 [==============================] - 0s 241ms/step
>11469, d1[0.148] d2[0.140] g[1.666]
1/1 [==============================] - 0s 198ms/step
>11470, d1[0.115] d2[0.153] g[2.866]
1/1 [==============================] - 0s 199ms/step
>11471, d1[0.303] d2[0.208] g[2.675]
1/1 [==============================] - 0s 277ms/step
>11472, d1[0.072] d2[0.093] g[2.701]
1/1 [==============================] - 0s 226ms/step
>11473, d1[0.113] d2[0.121] g[1.878]
1/1 [==============================] - 0s 209ms/step
>11474, d1[0.065] d2[0.140] g[1.605]
1/1 [=====